Отлично! Ниже — **быстрый универсальный toolkit для LLM-генерации**: один модуль с

* загрузкой локальных HF-моделей (offline-friendly),
* 8-бит/4-бит загрузкой (если есть bitsandbytes),
* шаблонами промптов под ChatML / Llama-2 / Mistral / Qwen / Gemma,
* единым методом `generate()` и батч-версией,
* стоп-последовательностями, температурой, top-p/top-k, повторениями,
* стриминговой генерацией (итератор),
* простым few-shot, а также JSON-ориентированной заготовкой,
* мини-примером использования.

Формат как ты просил: **код + короткие HTML-вставки с пояснениями**. Хочешь — удали HTML, останется чистый Python.

---

## 🚀 Универсальный модуль LLM (офлайн, кластер-дружелюбный)

````python
# ==============================
# LLM UNIVERSAL GENERATION TOOL
# ==============================

import os, sys, math, time, gc, warnings, pathlib, random
from typing import List, Dict, Any, Optional, Iterable, Generator, Tuple, Union

import torch

warnings.filterwarnings("ignore")


# ---------- Утилиты: устройство и seed ----------
def get_device() -> torch.device:
    if torch.cuda.is_available():
        return torch.device("cuda")
    if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
        return torch.device("mps")  # Apple Silicon
    return torch.device("cpu")

DEVICE = get_device()

def seed_everything(seed: int = 42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(42)


# ---------- Промпт-шаблоны под разные чаты ----------
class ChatPrompt:
    """
    Универсальный сборщик промптов.
    Поддерживает несколько стилей: 'chatml', 'llama2', 'mistral', 'qwen', 'gemma', 'raw'
    """

    @staticmethod
    def chatml(system: str, user: str, shots: List[Tuple[str, str]] = None) -> str:
        # OpenAI/ChatML-стиль
        parts = []
        if system:
            parts += ["<|im_start|>system\n" + system + "\n<|im_end|>"]
        if shots:
            for s_u, s_a in shots:
                parts += ["<|im_start|>user\n" + s_u + "\n<|im_end|>",
                          "<|im_start|>assistant\n" + s_a + "\n<|im_end|>"]
        parts += ["<|im_start|>user\n" + user + "\n<|im_end|>",
                  "<|im_start|>assistant\n"]
        return "\n".join(parts)

    @staticmethod
    def llama2(system: str, user: str, shots: List[Tuple[str, str]] = None) -> str:
        sys_block = f"<<SYS>>\n{system}\n<</SYS>>\n\n" if system else ""
        shot_text = ""
        if shots:
            for s_u, s_a in shots:
                shot_text += f"[INST] {sys_block}{s_u} [/INST] {s_a}\n"
                sys_block = ""  # системка только в первом блоке
        return f"{shot_text}[INST] {sys_block}{user} [/INST]"

    @staticmethod
    def mistral(system: str, user: str, shots: List[Tuple[str, str]] = None) -> str:
        # токены <s>[INST] ... [/INST] ответ
        sys_block = f"{system}\n\n" if system else ""
        shot_text = ""
        if shots:
            for s_u, s_a in shots:
                shot_text += f"<s>[INST] {sys_block}{s_u} [/INST] {s_a}</s>\n"
                sys_block = ""
        return f"{shot_text}<s>[INST] {sys_block}{user} [/INST]"

    @staticmethod
    def qwen(system: str, user: str, shots: List[Tuple[str, str]] = None) -> str:
        # просто ChatML-подобный стиль работает ок для Qwen-Instruct
        return ChatPrompt.chatml(system, user, shots)

    @staticmethod
    def gemma(system: str, user: str, shots: List[Tuple[str, str]] = None) -> str:
        # простой instruct подойдёт
        prompt = ""
        if system:
            prompt += f"System: {system}\n\n"
        if shots:
            for s_u, s_a in shots:
                prompt += f"User: {s_u}\nAssistant: {s_a}\n"
        prompt += f"User: {user}\nAssistant:"
        return prompt

    @staticmethod
    def raw(system: str, user: str, shots: List[Tuple[str, str]] = None) -> str:
        # Прямой конкат без специальных токенов (универсальный fallback)
        parts = []
        if system:
            parts += [f"[SYSTEM]\n{system}\n"]
        if shots:
            for s_u, s_a in shots:
                parts += [f"[USER]\n{s_u}\n[ASSISTANT]\n{s_a}\n"]
        parts += [f"[USER]\n{user}\n[ASSISTANT]\n"]
        return "\n".join(parts)

    @staticmethod
    def build(style: str, system: str, user: str, shots: Optional[List[Tuple[str, str]]] = None) -> str:
        style = (style or "raw").lower()
        if   style == "chatml":  return ChatPrompt.chatml(system, user, shots)
        elif style == "llama2":  return ChatPrompt.llama2(system, user, shots)
        elif style == "mistral": return ChatPrompt.mistral(system, user, shots)
        elif style == "qwen":    return ChatPrompt.qwen(system, user, shots)
        elif style == "gemma":   return ChatPrompt.gemma(system, user, shots)
        else:                    return ChatPrompt.raw(system, user, shots)


# ---------- Загрузка модели через transformers (с 8/4-бит, если доступно) ----------
class LLM:
    """
    Универсальная обёртка над HuggingFace transformers для локальной генерации.
    Поддержка: fp16/bf16, 8-бит/4-бит (bitsandbytes), device_map="auto", stop-seq, стриминг.
    """

    def __init__(
        self,
        model_path: str,
        dtype: str = "auto",            # "auto" | "fp16" | "bf16" | "fp32"
        load_in_8bit: bool = False,
        load_in_4bit: bool = False,
        device_map: str = "auto",
        trust_remote_code: bool = True,
        use_fast_tokenizer: bool = True,
        attn_implementation: Optional[str] = None,  # "flash_attention_2" если собран
    ):
        from transformers import AutoModelForCausalLM, AutoTokenizer

        self.model_path = model_path
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_path,
            use_fast=use_fast_tokenizer,
            trust_remote_code=trust_remote_code
        )

        # выбор dtype
        torch_dtype = None
        if dtype == "fp16": torch_dtype = torch.float16
        elif dtype == "bf16": torch_dtype = torch.bfloat16
        elif dtype == "fp32": torch_dtype = torch.float32
        else: torch_dtype = None  # auto

        quant_kwargs = {}
        if load_in_8bit or load_in_4bit:
            try:
                import bitsandbytes as bnb  # noqa
                if load_in_8bit: quant_kwargs["load_in_8bit"] = True
                if load_in_4bit:
                    quant_kwargs["load_in_4bit"] = True
                    quant_kwargs["bnb_4bit_use_double_quant"] = True
                    quant_kwargs["bnb_4bit_compute_dtype"] = torch.bfloat16
            except Exception as e:
                print("[warn] bitsandbytes недоступен, загружаю без квантования.")

        extra = {}
        if attn_implementation:
            extra["attn_implementation"] = attn_implementation

        self.model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch_dtype,
            device_map=device_map,
            trust_remote_code=trust_remote_code,
            **quant_kwargs,
            **extra,
        )
        self.model.eval()

        # токенизатор: паддинг/тримминг
        if self.tokenizer.pad_token is None:
            # безопасный пад-токен
            self.tokenizer.pad_token = self.tokenizer.eos_token or "<|endoftext|>"
        self.tokenizer.padding_side = "left"  # удобно для генерации разной длины (batch)

    # -------- единичная генерация --------
    @torch.no_grad()
    def generate(
        self,
        prompt: str,
        max_new_tokens: int = 256,
        temperature: float = 0.7,
        top_p: float = 0.9,
        top_k: int = 0,
        repetition_penalty: float = 1.05,
        no_repeat_ngram_size: int = 0,
        do_sample: bool = True,
        stop: Optional[List[str]] = None,
        stream: bool = False,
        **gen_kwargs,
    ) -> Union[str, Generator[str, None, str]]:
        """
        Возвращает строку (или генератор строк при stream=True).
        Параметры совместимы с transformers.generate().
        """
        # токенизация
        inputs = self.tokenizer([prompt], return_tensors="pt").to(self.model.device)

        gen_config = dict(
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=do_sample,
            repetition_penalty=repetition_penalty,
            no_repeat_ngram_size=no_repeat_ngram_size,
            pad_token_id=self.tokenizer.pad_token_id,
            eos_token_id=self.tokenizer.eos_token_id,
        )
        if top_k > 0:
            gen_config["top_k"] = top_k
        gen_config.update(gen_kwargs)

        if not stream:
            output_ids = self.model.generate(**inputs, **gen_config)
            text = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)
            # вырежем префикс (prompt) если нужно
            if text.startswith(prompt):
                text = text[len(prompt):]
            text = self._apply_stop(text, stop)
            return text.strip()

        # потоковый режим — итеративно растим вывод
        return self._stream_generate(prompt, inputs, gen_config, stop)

    def _apply_stop(self, text: str, stop: Optional[List[str]]) -> str:
        if not stop:
            return text
        cut = len(text)
        for s in stop:
            if not s:
                continue
            idx = text.find(s)
            if idx != -1:
                cut = min(cut, idx)
        return text[:cut]

    def _stream_generate(
        self, prompt: str, inputs, gen_config: Dict[str, Any], stop: Optional[List[str]]
    ) -> Generator[str, None, str]:
        """
        Простой потоковый вывод: по токену/фразе.
        """
        # Используем greedy "manually" токен за токеном (для стабильности в офлайне).
        # Для продвинутого стриминга можно подключить TextIteratorStreamer из transformers.
        from transformers import LogitsProcessorList

        input_ids = inputs["input_ids"]
        generated = input_ids.clone()

        logits_processors = LogitsProcessorList()
        max_new = int(gen_config.get("max_new_tokens", 128))
        temperature = float(gen_config.get("temperature", 1.0))
        top_p = float(gen_config.get("top_p", 1.0))
        top_k = int(gen_config.get("top_k", 0))
        repetition_penalty = float(gen_config.get("repetition_penalty", 1.0))
        eos_id = gen_config.get("eos_token_id", self.tokenizer.eos_token_id)

        decoded = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
        acc = ""  # только добавляемая часть (без префикса)

        for _ in range(max_new):
            outputs = self.model(input_ids=generated)
            logits = outputs.logits[:, -1, :]  # [1, vocab]
            # repetition penalty
            if repetition_penalty != 1.0:
                for token_id in set(generated[0].tolist()):
                    logits[:, token_id] /= repetition_penalty
            # softmax с temperature
            if temperature > 0:
                logits = logits / temperature

            probs = torch.softmax(logits, dim=-1)

            # nucleus/top-k
            if top_k > 0:
                values, indices = torch.topk(probs, top_k)
                probs_zero = torch.zeros_like(probs)
                probs_zero.scatter_(1, indices, values)
                probs = probs_zero
            if top_p < 1.0:
                sorted_probs, sorted_idx = torch.sort(probs, descending=True)
                cumsum = torch.cumsum(sorted_probs, dim=-1)
                mask = cumsum - sorted_probs > top_p
                sorted_probs[mask] = 0
                probs = torch.zeros_like(probs).scatter(1, sorted_idx, sorted_probs)
            probs /= probs.sum(dim=-1, keepdim=True)

            next_token = torch.multinomial(probs, num_samples=1)  # [1,1]
            generated = torch.cat([generated, next_token], dim=1)

            if next_token.item() == eos_id:
                break

            new_text = self.tokenizer.decode(next_token[0], skip_special_tokens=True)
            acc += new_text
            # проверка стоп-последовательностей
            if stop and any(st in acc for st in stop if st):
                acc = self._apply_stop(acc, stop)
                yield acc
                return acc
            yield acc

        return acc


# ---------- Вспомогательные генераторы запросов ----------
def format_prompt(
    system: str,
    user: str,
    style: str = "chatml",
    shots: Optional[List[Tuple[str, str]]] = None
) -> str:
    """
    Сборка промпта под указанный стиль (chatml/llama2/mistral/qwen/gemma/raw).
    """
    return ChatPrompt.build(style=style, system=system, user=user, shots=shots)


# ---------- JSON-ориентированная заготовка ----------
def json_guardrail(text: str) -> str:
    """
    Очень простой guardrail: обрезать до первого '```json'...'```' или фигурные скобки.
    Для серьёзной валидации — jsonschema / pydantic + итеративный repair.
    """
    t = text.strip()
    start = t.find("{")
    end = t.rfind("}")
    if start != -1 and end != -1 and end > start:
        return t[start:end+1]
    return t
````

```markdown
<div>
<b>Что внутри модуля:</b><br>
• <code>LLM</code> — единая обёртка над transformers: локальная загрузка, квантование 8/4-бит (если доступно), fp16/bf16, device_map="auto"<br>
• <code>ChatPrompt</code> — сборщик промптов под популярные шаблоны (ChatML/Llama2/Mistral/Qwen/Gemma)<br>
• <code>generate(...)</code> — единичный вызов; <code>stream=True</code> вернёт генератор (построчная подача вывода)<br>
• <code>format_prompt(...)</code> — удобная сборка текста запроса<br>
• <code>json_guardrail(...)</code> — примитивный “резак” до валидного JSON-блока (буст для structured output)
</div>
```

---

## 🧪 Мини-пример: обычная генерация

```python
if __name__ == "__main__":
    # Пример — подставь свою локальную модель (папка/путь):
    # Часто на кластерах пути вида: "/home/jovyan/models/Qwen2.5-7B-Instruct"
    MODEL_PATH = os.environ.get("LLM_PATH", "YOUR_LOCAL_INSTRUCT_MODEL")

    llm = LLM(
        model_path=MODEL_PATH,
        dtype="bf16",          # auto|fp16|bf16|fp32
        load_in_4bit=False,    # True если нужен экономный режим (bitsandbytes)
        device_map="auto",
        trust_remote_code=True
    )

    system = "Ты — полезный, краткий, честный ассистент."
    user = "Объясни разницу между LightGBM и XGBoost простым языком, в 5 пунктах."
    prompt = format_prompt(system=system, user=user, style="chatml")

    # 1) Обычная генерация
    out = llm.generate(
        prompt,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.05,
        stop=["<|im_end|>", "</s>"]
    )
    print("=== Ответ (обычный) ===")
    print(out)

    # 2) Стриминг (постепенный вывод)
    print("\n=== Ответ (stream) ===")
    stream_iter = llm.generate(prompt, max_new_tokens=128, temperature=0.7, stream=True)
    final = ""
    for chunk in stream_iter:
        print(chunk, end="", flush=True)
        final = chunk
    print("\n[END STREAM]")
```

```markdown
<div>
<b>Советы:</b><br>
• Если ответ “жует” ввод (повторяет prompt), просто вырезай префикс (в коде уже предусмотрено).<br>
• Для детерминизма: <code>temperature=0, top_p=1, do_sample=False</code>.<br>
• Для разнообразия: <code>temperature≈0.8–1.0, top_p≈0.9–0.95</code>.
</div>
```

---

## 📦 Батч-генерация (списки запросов)

```python
def batch_generate(
    llm: LLM,
    prompts: List[str],
    max_new_tokens: int = 256,
    temperature: float = 0.7,
    top_p: float = 0.9,
    top_k: int = 50,
    repetition_penalty: float = 1.05,
    stop: Optional[List[str]] = None,
    **gen_kwargs
) -> List[str]:
    """
    Простая батчевая генерация.
    Вариант 1: циклом (чтобы не ловить OOM).
    Вариант 2: один вызов .generate() на объединённом батче (сложнее из-за разной длины).
    """
    outs = []
    for p in prompts:
        text = llm.generate(
            p,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=repetition_penalty,
            stop=stop,
            **gen_kwargs
        )
        outs.append(text)
    return outs

# Пример:
# prompts = [format_prompt("Система","Сделай 3 идеи стартапа для школы", style="mistral") for _ in range(10)]
# answers = batch_generate(llm, prompts)
```

---

## 🧩 Few-shot: короткая схема

```python
shots = [
    ("Что такое градиент?", "Градиент — это вектор частных производных, указывающий направление наибольшего роста функции."),
    ("А что такое шаг градиентного спуска?", "Это величина изменения параметров в направлении антиградиента на каждой итерации.")
]

prompt_fewshot = format_prompt(
    system="Отвечай кратко и понятно.",
    user="Объясни, что такое переобучение и как его избежать?",
    style="llama2",
    shots=shots
)

# out = llm.generate(prompt_fewshot, max_new_tokens=200)
# print(out)
```

---

## 🧰 Структурированный ответ (JSON-намёк)

```python
user = (
    "Сформируй JSON с полями: "
    "title (строка), pros (список), cons (список) по модели LightGBM."
)

prompt_json = format_prompt(
    system="Отвечай ТОЛЬКО валидным JSON без комментариев и лишнего текста.",
    user=user,
    style="chatml"
)

text = llm.generate(prompt_json, max_new_tokens=200, temperature=0.2, top_p=0.9, do_sample=False)
json_like = json_guardrail(text)
print(json_like)
```

---

## ⚠️ Частые проблемы и что делать

```markdown
<div>
<ul>
<li><b>Нет интернета / офлайн:</b> используй <code>model_path</code> на локальную папку модели (веса скачай заранее). Отключай <code>trust_remote_code=False</code>, если код недоверенный.</li>
<li><b>CUDA OOM:</b> уменьши <code>max_new_tokens</code>, <code>top_k</code>, включи 4-бит (<code>load_in_4bit=True</code>), батчи генерируй по одному.</li>
<li><b>Модель отвечает медленно:</b> <code>dtype="bf16/fp16"</code>, <code>attn_implementation="flash_attention_2"</code> (если есть сборка), поменьше контекст, отключи лишние шоты.</li>
<li><b>Повторы и “залипание”:</b> увеличь <code>repetition_penalty</code> (1.1–1.3), <code>no_repeat_ngram_size=3–5</code>, <code>temperature≈0.8</code>.</li>
<li><b>Шум/галлюцинации:</b> уменьши <code>temperature</code> (0–0.5), <code>do_sample=False</code>, добавь строгую системную инструкцию.</li>
<li><b>Нужно строго по формату:</b> добавляй в системный промпт требования, используй post-processing (как <code>json_guardrail</code>), при необходимости несколько попыток (self-repair).</li>
</ul>
</div>
```

---

Если нужно — сделаю рядом **минимальную .py версию** без HTML, или добавлю **vLLM/TGI** обёртки под серверную генерацию.
